In [1]:
import torch
import torchvision
import torchmetrics
from torchvision import transforms
from torch import nn
from torch.utils.data import DataLoader

import wandb
import numpy as np
import matplotlib.pyplot as plt
from tqdm.auto import tqdm

from utils import *

In [ ]:
train_data = torchvision.datasets.EMNIST(
    root = "data",
    split = "balanced",
    train = True,
    download = True,
    transform = transforms.ToTensor()
)

test_data = torchvision.datasets.EMNIST(
    root = "data",
    split = "balanced",
    train = False,
    download = True,
    transform = transforms.ToTensor()
)

In [ ]:
wandb.login()
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
config = dict(
    epochs = 5,
    in_features = 1,
    classes = 47,
    kernels = [32, 64],
    batch_size = 32,
    learning_rate = 0.001,
    dataset="EMNIST",
    architecture="ON"
)

In [ ]:
class eMNISTModel(nn.Module):

    def __init__(self, input_features: int, kernels: list[int], output_features: int):
        super().__init__()
        self.conv_block1 = nn.Sequential(
            nn.Conv2d(in_channels=input_features, out_channels=kernels[0],
                      kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=kernels[0], out_channels=kernels[0],
                      kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        self.conv_block2 = nn.Sequential(
            nn.Conv2d(in_channels=kernels[0], out_channels=kernels[1],
                      kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=kernels[1], out_channels=kernels[1],
                      kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(in_features=kernels[-1]*49, out_features=output_features)
        )

    def forward(self, x: torch.tensor) -> torch.tensor:
        return self.classifier(self.conv_block2(self.conv_block1(x)))
    
    

In [ ]:
class eMNISTModelTraditional(nn.Module):
    
    def __init__(self, input_features:int, kernels: list[int], output_features: int):
        super().__init__()
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(in_features = input_features, out_features = kernels[0]),
            nn.ReLU(),
            nn.Linear(in_features = kernels[0], out_features = kernels[1]),
            nn.ReLU(),
            nn.Linear(in_features = kernels[1], out_features = output_features)
        )

    def forward(self, x: torch.tensor) -> torch.tensor:
        return self.classifier(x)

In [ ]:
model = model_pipeline("EMNIST", eMNISTModel(1, [32, 64], 47), train_data, test_data, config, device)

In [ ]:
class_names = test_data.classes
test_dataloader = DataLoader(test_data, batch_size=32, shuffle=False)

model.eval()
with torch.inference_mode():
    batch_data, batch_label = next(iter(test_dataloader))
    rand_idx = torch.randint(0, 32, (1,1)).item()
    rand_idx = 12
    img, label = batch_data[rand_idx], batch_label[rand_idx]
    print(img.shape)
    logit = model(img.unsqueeze(1).to(device))
    pred = torch.softmax(logit, dim=1).argmax(dim=1)
    plt.imshow(img.squeeze())
    plt.title(f"Act: {class_names[label]} | Pred: {class_names[pred]}")
